In [ ]:
import numpy as np
import time
import pandas

In [ ]:
def compute_mae(var: np.ndarray, var_hat: np.ndarray) -> float:
    """
    Compute the Mean Absolute Error (MAE) between two arrays.

    Args:
        var (np.ndarray): The true values.
        var_hat (np.ndarray): The predicted values.

    Returns:
        float: The MAE.
    """
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var: np.ndarray, var_hat: np.ndarray) -> float:
    """
    Compute the Root Mean Square Error (RMSE) between two arrays.

    Args:
        var (np.ndarray): The true values.
        var_hat (np.ndarray): The predicted values.

    Returns:
        float: The RMSE.
    """
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [ ]:
def laplacian(n: int, tau: int) -> np.ndarray:
    """
    Compute the discrete Laplacian operator.

    Args:
        n (int): The size of the Laplacian operator.
        tau (int): The parameter for the Laplacian.

    Returns:
        np.ndarray: The Laplacian operator.
    """
    ell = np.zeros(n)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z: np.ndarray, w: np.ndarray, lmbda: float, denominator: np.ndarray) -> np.ndarray:
    """
    Compute the proximal operator.

    Args:
        z (np.ndarray): Input array.
        w (np.ndarray): Input array.
        lmbda (float): Regularization parameter.
        denominator (np.ndarray): Denominator array.

    Returns:
        np.ndarray: Result of the proximal operator.
    """
    T = z.shape[0]
    temp = np.fft.fft(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft(temp * temp1).real

def update_z(y_train: np.ndarray, pos_train: np.ndarray, x: np.ndarray, w: np.ndarray, lmbda: float, eta: float) -> np.ndarray:
    """
    Update the variable z.

    Args:
        y_train (np.ndarray): The training data.
        pos_train (np.ndarray): Boolean array indicating positive training examples.
        x (np.ndarray): Input variable.
        w (np.ndarray): Weight variable.
        lmbda (float): Regularization parameter.
        eta (float): Learning rate parameter.

    Returns:
        np.ndarray: Updated z.
    """
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x: np.ndarray, z: np.ndarray, w: np.ndarray, lmbda: float) -> np.ndarray:
    """
    Update the variable w.

    Args:
        x (np.ndarray): Input variable.
        z (np.ndarray): Input variable.
        w (np.ndarray): Weight variable.
        lmbda (float): Regularization parameter.

    Returns:
        np.ndarray: Updated w.
    """
    return w + lmbda * (x - z)


In [ ]:
def LCR(y_true: np.ndarray, y: np.ndarray, lmbda: float, gamma: float, tau: int, maxiter: int = 50) -> np.ndarray:
    """
    LCR (Laplacian Constrained Regression) algorithm.

    Args:
        y_true (np.ndarray): The true target values.
        y (np.ndarray): The input values.
        lmbda (float): Regularization parameter.
        gamma (float): Gamma parameter.
        tau (int): Tau parameter.
        maxiter (int, optional): Maximum number of iterations. Default is 50.

    Returns:
        np.ndarray: The result of the LCR algorithm.
    """
    eta = 100 * lmbda
    T = y.shape[0]
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    denominator = lmbda + gamma * np.fft.fft(laplacian(T, tau)) ** 2
    T = y_true.shape[0]
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
    print(f'lambda: {lmbda/T}, gamma: {gamma/lmbda}, tau: {tau}')
    mae_accuracy = compute_mae(y_test, x[pos_test])
    rmse_accuracy = compute_rmse(y_test, x[pos_test])
    print(f'MAE Accuracy: {mae_accuracy}')
    print(f'RMSE Accuracy: {rmse_accuracy}')
    print(round(mae_accuracy, 5))
    print(round(rmse_accuracy, 5))
    print()
    return x

In [ ]:
batch_size=20
df = pandas.read_csv('dataset1.csv',nrows=batch_size)
missing_rate = 0.20
print('Missing rate = {}'.format(missing_rate))
start_time = time.monotonic()
dense_mat = df.to_numpy()
dense_vec = dense_mat[:,0]  
T = dense_vec.shape[0]
np.random.seed(1)
sparse_vec = dense_vec * np.round(np.random.rand(T) + 0.5 - missing_rate)
start_time = time.time()
lmbda = 5e-3 * T
gamma = 2 * lmbda
tau = 3
maxiter = 100
x = LCR(dense_vec, sparse_vec, lmbda, gamma, tau, maxiter)
end_time = time.monotonic()
elapsed_time_seconds = end_time - start_time
elapsed_time_ms = elapsed_time_seconds * 1000
print("--- %s seconds ---" % (time.time() - start_time))